# Celda 1 — Imports, semillas, hilos, rutas y utilidades

In [ ]:
# ==== Imports base ====
import os, time, gc, itertools, json
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import matplotlib.pyplot as plt

# ==== Reproducibilidad ====
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ==== Control de hilos (Windows/CPU) para estabilidad ====
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)

# ==== Rutas ====
DATA_DIR   = Path("../data/processed/seq")
REPORT_DIR = Path("../reports/models")
REPORT_DIR.mkdir(parents=True, exist_ok=True)

# util para tiempo bonito
def mmss(t):
    m = int(t // 60); s = int(t % 60)
    return f"{m:02d}:{s:02d}"


# Celda 2 — Carga de secuencias (de Cuaderno 4)

In [ ]:
def load_seq(ticker: str, window: int):
    base = DATA_DIR / ticker / f"w{window}"
    Xtr = np.load(base/"X_train.npy"); ytr = np.load(base/"y_train.npy"); idx_tr = pd.DatetimeIndex(np.load(base/"idx_train.npy"))
    Xva = np.load(base/"X_val.npy");   yva = np.load(base/"y_val.npy");   idx_va = pd.DatetimeIndex(np.load(base/"idx_val.npy"))
    Xte = np.load(base/"X_test.npy");  yte = np.load(base/"y_test.npy");  idx_te = pd.DatetimeIndex(np.load(base/"idx_test.npy"))
    meta = json.load(open(base/"meta.json","r"))
    return (Xtr,ytr),(Xva,yva),(Xte,yte),meta

# Prueba rápida (opcional; puedes comentar si ya lo sabes)
for tkr in ["BBVA","SAN"]:
    for W in [10,20,30]:
        (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, W)
        print(f"{tkr} W={W} -> Xtr{Xtr.shape}, Xva{Xva.shape}, Xte{Xte.shape}, feats={meta['n_features']}")


# Celda 3 — Baseline de persistencia (diagnóstico)

In [ ]:
def naive_persistence(X3: np.ndarray) -> np.ndarray:
    """Devuelve el último Close de la ventana (asumimos Close como 1ª feature)."""
    return X3[:, -1, 0]

for tkr in ["BBVA","SAN"]:
    (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, 20)
    yhat_naive = naive_persistence(Xte)
    mse_naive = np.mean((yhat_naive - yte)**2)
    print(f"{tkr} · Naive (persistencia) · MSE test = {mse_naive:.6f}")


# Celda 4 — Constructor de modelos (robusto y sin warning)

In [ ]:
def build_model(model_type: str, units: int, n_features: int, window_size: int, lr: float) -> keras.Model:
    inp = keras.Input(shape=(window_size, n_features), name="seq")
    if model_type == "SimpleRNN":
        x = layers.SimpleRNN(units, return_sequences=True, dropout=0.1, recurrent_dropout=0.1,
                             kernel_regularizer=regularizers.l2(1e-5))(inp)
        x = layers.SimpleRNN(units, dropout=0.1, recurrent_dropout=0.1)(x)
    elif model_type == "LSTM":
        x = layers.LSTM(units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        x = layers.LSTM(units, dropout=0.2, recurrent_dropout=0.2)(x)
    elif model_type == "GRU":
        x = layers.GRU(units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(inp)
        x = layers.GRU(units, dropout=0.2, recurrent_dropout=0.2)(x)
    else:
        raise ValueError("model_type debe ser 'SimpleRNN', 'LSTM' o 'GRU'")
    out = layers.Dense(1, name="y")(x)

    model = keras.Model(inp, out, name=f"{model_type}_u{units}_w{window_size}")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mse"])
    return model


# Celda 5 — Callbacks de progreso y límite por combo

In [ ]:
MAX_EPOCHS = 80     # puedes ajustar
PATIENCE   = 3       # EarlyStopping
MAX_SECS   = 30*60   # ⏱️ límite por combo (None para sin límite)

class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.t0 = time.time()
    def on_epoch_begin(self, epoch, logs=None):
        print(f"    · epoch {epoch+1}/{self.params.get('epochs', '?')} ...", flush=True)
    def on_train_end(self, logs=None):
        print(f"    · done in {mmss(time.time()-self.t0)}", flush=True)

class StopAfterSeconds(keras.callbacks.Callback):
    def __init__(self, seconds=None):
        super().__init__(); self.seconds = seconds
    def on_train_begin(self, logs=None):
        self.t0 = time.time()
    def on_epoch_end(self, epoch, logs=None):
        if self.seconds is not None and (time.time()-self.t0) > self.seconds:
            print("    ⏱️  Paro por tiempo máximo de combo", flush=True)
            self.model.stop_training = True


# Celda 6 — Entrenar con logs + evaluar (sustituye a fit_and_eval)

In [ ]:
def fit_and_eval_with_logs(model_type, tkr, W, units, batch, lr, epochs=MAX_EPOCHS):
    # Datos
    (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, W)
    n_features = meta["n_features"]

    # Modelo
    model = build_model(model_type, units, n_features, W, lr)

    # Datasets tf.data (más estables en memoria)
    ds_tr = tf.data.Dataset.from_tensor_slices((Xtr, ytr)).batch(batch).cache().prefetch(tf.data.AUTOTUNE)
    ds_va = tf.data.Dataset.from_tensor_slices((Xva, yva)).batch(batch)
    ds_te = tf.data.Dataset.from_tensor_slices((Xte, yte)).batch(batch)

    # Callbacks
    cbs = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
           TimeHistory(),
           StopAfterSeconds(MAX_SECS)]

    # Entrenamiento
    h = model.fit(ds_tr, validation_data=ds_va, epochs=epochs, verbose=0, callbacks=cbs)

    # Evaluación
    y_pred_va = model.predict(ds_va, verbose=0).ravel()
    y_pred_te = model.predict(ds_te, verbose=0).ravel()
    val_mse = float(np.mean((y_pred_va - yva)**2))
    test_mse = float(np.mean((y_pred_te - yte)**2))

    # Limpieza memoria
    keras.backend.clear_session(); del model
    gc.collect()

    return {"val_mse": val_mse, "test_mse": test_mse}


# Celda 7 — Configuración del grid (con recorte SAN·LSTM)

In [ ]:
TICKERS = ["BBVA", "SAN"]
MODELS  = ["SimpleRNN", "LSTM", "GRU"]

GRID = {
    "window": [10,20,30],
    "units":  [32,64,128],
    "batch":  [32,64],
    "lr":     [1e-3, 5e-4],
}

def grid_for(ticker, model):
    g = {k: v[:] for k, v in GRID.items()}
    # ⚠️ Cuello típico: SAN · LSTM — lo recortamos para no atascar
    if ticker == "SAN" and model == "LSTM":
        g["window"] = [20,30]
        g["units"]  = [64,128]
        g["batch"]  = [32]   # quita 64 para evitar swap
    return g


# Celda 8 — (Opcional) Curvas de pérdida rápidas (5/7/10 epochs)

In [ ]:
BASE_CFG = dict(units=64, batch=32, lr=1e-3)
EPOCHS_LIST = [5,7,10]
WINDOW_FOR_PLOTS = 20

def train_history(model_type, tkr, W, units, batch, lr, epochs):
    (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, W)
    model = build_model(model_type, units, meta["n_features"], W, lr)
    es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
    h = model.fit(Xtr, ytr, validation_data=(Xva,yva), epochs=epochs, batch_size=batch, verbose=0, callbacks=[es])
    hist = {"loss": h.history["loss"], "val_loss": h.history.get("val_loss", [])}
    keras.backend.clear_session(); del model; gc.collect()
    return hist

def plot_losses(histories, title):
    plt.figure(figsize=(8,5))
    for ep, h in histories.items():
        plt.plot(h["loss"], label=f"train (e={ep})")
        if len(h["val_loss"]):
            plt.plot(h["val_loss"], label=f"val (e={ep})", linestyle="--")
    plt.title(title); plt.xlabel("Época"); plt.ylabel("MSE (pérdida)")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

for tkr in TICKERS:
    for m in MODELS:
        hist_dict = {}
        for ep in EPOCHS_LIST:
            hist_dict[ep] = train_history(m, tkr, WINDOW_FOR_PLOTS, **BASE_CFG, epochs=ep)
        plot_losses(hist_dict, f"{tkr} · {m} — Curvas de pérdida (W={WINDOW_FOR_PLOTS})")


# Celda 9 — Grid Search con progreso, guardado incremental y reanudación

In [ ]:
OUT_DIR = REPORT_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)

def grid_search_for_model(tkr: str, model_type: str) -> pd.DataFrame:
    partial_csv = OUT_DIR / f"grid_partial_{tkr}_{model_type}.csv"
    rows = []

    # Reanudación si existe parcial
    if partial_csv.exists():
        prev = pd.read_csv(partial_csv)
        rows = prev.to_dict("records")
        print(f"⏭️  Reanudando: {len(rows)} combos ya terminados")

    done = {(r["window"], r["units"], r["batch"], float(r["lr"])) for r in rows}
    g = grid_for(tkr, model_type)
    combos = list(itertools.product(g["window"], g["units"], g["batch"], g["lr"]))
    total = len(combos)

    for i, (W, U, B, LR) in enumerate(combos, start=1):
        if (W,U,B,float(LR)) in done:
            print(f"⏭️  {tkr} · {model_type} · combo {i}/{total} (W={W},U={U},B={B},lr={LR}) ya estaba")
            continue

        print(f"\n▶ {tkr} · {model_type} · combo {i}/{total}  (W={W}, U={U}, B={B}, lr={LR})")
        t0 = time.time()
        try:
            res = fit_and_eval_with_logs(model_type, tkr, W, units=U, batch=B, lr=LR, epochs=MAX_EPOCHS)
            status = "ok"
        except Exception as e:
            print(f"    ⚠️  Error: {type(e).__name__}: {e}")
            res = {"val_mse": np.nan, "test_mse": np.nan}
            status = f"error:{type(e).__name__}"

        dt = time.time() - t0
        print(f"    ↳ val_mse={res['val_mse']:.6f} · test_mse={res['test_mse']:.6f} · {mmss(dt)}")

        rows.append({
            "ticker": tkr, "model": model_type,
            "window": W, "units": U, "batch": B, "lr": LR,
            "val_mse": res["val_mse"], "test_mse": res["test_mse"],
            "secs": round(dt,1), "status": status
        })
        pd.DataFrame(rows).to_csv(partial_csv, index=False)

        time.sleep(0.2)  # respiro
        gc.collect()

    df_rank = pd.DataFrame(rows).sort_values(["val_mse","test_mse"], na_position="last").reset_index(drop=True)
    return df_rank

all_ranks = []
for tkr in TICKERS:
    for m in MODELS:
        print(f"\n==== Grid Search: {tkr} · {m} ====")
        rank_df = grid_search_for_model(tkr, m)
        rank_df.to_csv(OUT_DIR / f"grid_{tkr}_{m}.csv", index=False)
        all_ranks.append(rank_df.assign(order=range(1, len(rank_df)+1)))

grid_all = pd.concat(all_ranks, ignore_index=True)
grid_all.head()


# Celda 10 — Top-10 por ticker/modelo y resumen “mejor de cada uno”

In [ ]:
# Top-10 por grupo
tops = []
for tkr in TICKERS:
    for m in MODELS:
        dfm = grid_all[(grid_all["ticker"]==tkr) & (grid_all["model"]==m)].sort_values(["val_mse","test_mse"]).head(10)
        dfm.to_csv(OUT_DIR / f"top10_{tkr}_{m}.csv", index=False)
        print(f"\n=== {tkr} · {m} — Top-10 (por val_mse) ===")
        display(dfm)
        tops.append(dfm.assign(kind=f"{tkr}_{m}"))

top_all = pd.concat(tops, ignore_index=True)
top_all.to_csv(OUT_DIR / "top10_all.csv", index=False)

# Mejor de cada grupo
best_rows = []
for t in TICKERS:
    for m in MODELS:
        best_rows.append(grid_all[(grid_all.ticker==t)&(grid_all.model==m)].sort_values(["val_mse","test_mse"]).iloc[0])
best_table = pd.DataFrame(best_rows).reset_index(drop=True)
best_table.to_csv(OUT_DIR / "best_models_summary.csv", index=False)
display(best_table)
print("✅ Guardado resumen:", OUT_DIR / "best_models_summary.csv")


# Celda 11 — Real vs Predicho de los mejores (plots)

In [ ]:
def best_config(df: pd.DataFrame, tkr: str, m: str):
    sub = df[(df["ticker"]==tkr) & (df["model"]==m)].sort_values(["val_mse","test_mse"]).head(1).iloc[0]
    return dict(W=int(sub["window"]), U=int(sub["units"]), B=int(sub["batch"]), LR=float(sub["lr"]))

def plot_real_pred(y_true, y_pred, title):
    plt.figure(figsize=(12,5))
    plt.plot(y_true, label="Real (Close t+1)", linewidth=1.2)
    plt.plot(y_pred, label="Predicho", linewidth=1.2)
    plt.title(title); plt.xlabel("Índice temporal (test)"); plt.ylabel("Close (escalado)")
    plt.grid(alpha=0.3); plt.legend(); plt.tight_layout(); plt.show()

best_runs = []
for tkr in TICKERS:
    for m in MODELS:
        cfg = best_config(grid_all, tkr, m)
        # Entrenamos SOLO para graficar (pocas épocas por rapidez)
        (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(tkr, cfg["W"])
        model = build_model(m, cfg["U"], meta["n_features"], cfg["W"], cfg["LR"])
        es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
        model.fit(Xtr, ytr, validation_data=(Xva,yva), epochs=min(20, MAX_EPOCHS), batch_size=cfg["B"], verbose=0, callbacks=[es])
        yhat = model.predict(Xte, verbose=0).ravel()
        val_mse = float(np.mean((model.predict(Xva, verbose=0).ravel() - yva)**2))
        test_mse = float(np.mean((yhat - yte)**2))
        plot_real_pred(yte, yhat, f"{tkr} · {m} — Mejor configuración (test)")
        best_runs.append({
            "ticker": tkr, "model": m, "window": cfg["W"], "units": cfg["U"],
            "batch": cfg["B"], "lr": cfg["LR"], "val_mse": val_mse, "test_mse": test_mse
        })
        keras.backend.clear_session(); del model; gc.collect()

best_table = pd.DataFrame(best_runs).sort_values(["ticker","test_mse"])
best_table.to_csv(OUT_DIR / "best_models_summary.csv", index=False)
display(best_table)


# Celda 12 — Mejora vs persistencia para los mejores

In [ ]:
def mse(a,b): return float(np.mean((a-b)**2))

for row in best_table.itertuples(index=False):
    (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(row.ticker, int(row.window))
    model = build_model(row.model, int(row.units), meta["n_features"], int(row.window), float(row.lr))
    es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
    model.fit(Xtr, ytr, validation_data=(Xva,yva), epochs=min(30, MAX_EPOCHS), batch_size=int(row.batch), verbose=0, callbacks=[es])
    yhat = model.predict(Xte, verbose=0).ravel()
    ynaive = naive_persistence(Xte)
    mse_model = mse(yte, yhat)
    mse_naive = mse(yte, ynaive)
    imp = 100*(1 - mse_model/mse_naive)
    print(f"{row.ticker} · {row.model} (W={row.window}, U={row.units}) → MSE={mse_model:.6f} | Naive={mse_naive:.6f} | Mejora={imp:.2f}%")
    keras.backend.clear_session(); del model; gc.collect()


# Celda 13 — Comparativa MSE (barras)

In [ ]:
def bar_compare(best_table, ticker):
    sub = best_table[best_table["ticker"]==ticker].sort_values("test_mse")
    plt.figure(figsize=(6,4))
    plt.bar(sub["model"], sub["test_mse"])
    for i,(m,v) in enumerate(zip(sub["model"], sub["test_mse"])):
        plt.text(i, v, f"{v:.5f}", ha="center", va="bottom", fontsize=9)
    plt.title(f"{ticker} — MSE (test) mejores modelos")
    plt.ylabel("MSE (test)"); plt.grid(axis="y", alpha=0.2)
    plt.tight_layout(); plt.show()

for tkr in TICKERS:
    bar_compare(best_table, tkr)


# Celda 14 — Guardar modelos .h5 (para Cuaderno 6)

In [ ]:
SAVE_MODELS = True

if SAVE_MODELS:
    for row in best_table.itertuples(index=False):
        (Xtr,ytr),(Xva,yva),(Xte,yte),meta = load_seq(row.ticker, int(row.window))
        model = build_model(row.model, int(row.units), meta["n_features"], int(row.window), float(row.lr))
        es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)
        model.fit(Xtr, ytr, validation_data=(Xva,yva), epochs=min(30, MAX_EPOCHS), batch_size=int(row.batch), verbose=0, callbacks=[es])
        path = OUT_DIR / f"{row.ticker}_{row.model}_w{row.window}_u{row.units}_b{row.batch}_lr{row.lr}.h5"
        model.save(path)
        print("💾 Guardado:", path)
        keras.backend.clear_session(); del model; gc.collect()
